<a href="https://colab.research.google.com/github/costiash/Fast-Style-Transfer/blob/main/StyleTransfer_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi all and welcome to my Style Transfer Demo :)

Style Transfer is a technique in which we use a convolutional neural network for transferring the "style" of some image into a "regular" 
content image. The model in this demo was made and trained by me, based on this 
<a href="https://arxiv.org/pdf/1603.08155.pdf">paper</a>.

INSTRUCTIONS FOR THE DEMO:



1.   Run the cells by the order they appear.
2.   Run each cell alone and wait for it to complete its run before you move to the next cell.

1.   Before you start, press the 'Connect' in the upper-right side. If you don't see it and instead you are seeing a green 'V' sign you are all good.
2.   For running each cell just press the 'play' button that appears on the left side of the cell.

1.   Read the individual instructions of each cell before you run it.
2.   HAVE FUN AND ENJOY!!!!

In [ ]:
#@title 1. Setup
#@markdown Just run this cell as is. ***Don't modify the code block.*** Notice that at some point you will be asked to enter a choice into the input box, just write '1' (the number one) in the input box and press enter.
#@markdown Make sure that the cell finish to run and the only output you see is "You are all set!" before you continue to the next cell. 

from IPython.display import clear_output
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
 
!pip install wandb -qq
import wandb
wandb.login(anonymous='allow')
clear_output()
content_image = None # This needs to be in global scope

print('You are all set!')

In [ ]:
#@title 2. Upload an image you want to stylize
#@markdown First, run the cell then press the 'Upload' button and finally press 'Show Image'. Wait until you see the image you just uploaded on the screen before you continue to the next cell.

#@markdown It's recommended to upload high resolution images for better effect.
from IPython.display import display
import ipywidgets as widgets
from PIL import Image
import numpy as np
import io

clear_output()

def load_image(image_path,dim=None,resize=False):
    img= Image.open(image_path)
    if dim:
        if resize:
            img=img.resize(dim,Image.LANCZOS)
        else:
            img.thumbnail(dim)
    img= img.convert("RGB")
    return np.array(img)


def show_image(image,title=None):
    if len(image.shape)>3:
        image=tf.squeeze(image,axis=0)
    plt.imshow(image)
    if title:
        plt.title=title


def button_click(change):
    global content_image
    print('\n')
    img = load_image(io.BytesIO(uploader.data[-1]))
    content_image = img
    show_image(content_image)
    plt.imsave('content.jpg', content_image)
    
uploader = widgets.FileUpload()
show_button = widgets.Button(description='Show image')
show_button.on_click(button_click)

widgets.VBox([widgets.Label('Upload a content image (jpg, jpeg or png).'), uploader, show_button])

In [ ]:
#@title 3. Choose a style image from the options below
#@markdown Press the little arrow in the right corner(in the next line) to see the list of styles and then choose the one you liked and run the cell. If you want to choose a different style afterword, make sure you run this cell (and the cells after it) again.
from urllib.parse import urlsplit
import urllib.request


STYLE_IMAGE_NAME = 'STYLE_1' #@param ['STYLE_1', 'STYLE_2', 'STYLE_3', 'STYLE_4', 'STYLE_5', 'STYLE_6', 'STYLE_7', 'STYLE_8', 'STYLE_9']

corresponding_url = {
    'STYLE_1': 'https://storage.googleapis.com/my-saved-models/style_transfer1.tflite',
    'STYLE_2': 'https://storage.googleapis.com/my-saved-models/style_transfer3.tflite',
    'STYLE_3': 'https://storage.googleapis.com/my-saved-models/style_transfer2.tflite',
    'STYLE_4': 'https://storage.googleapis.com/my-saved-models/style_transfer4.tflite',
    'STYLE_5': 'https://storage.googleapis.com/my-saved-models/style_transfer5.tflite',
    'STYLE_6': 'https://storage.googleapis.com/my-saved-models/style_transfer6.tflite',
    'STYLE_7': 'https://storage.googleapis.com/my-saved-models/style_transfer7.tflite',
    'STYLE_8': 'https://storage.googleapis.com/my-saved-models/style_transfer8.tflite',
    'STYLE_9': 'https://storage.googleapis.com/my-saved-models/style_transfer9.tflite',
}

url = corresponding_url[STYLE_IMAGE_NAME]
r1 = urlsplit(url)
style_transfer_path = '/content/style_transfer' + STYLE_IMAGE_NAME.split('_')[1] + '.tflite'
urllib.request.urlretrieve(r1.geturl(), style_transfer_path)
# style_image_path = tf.keras.utils.get_file(STYLE_IMAGE_NAME + ".jpg", corresponding_url[STYLE_IMAGE_NAME])
print("The Style Transfer Network is ready!")

<center><img src="https://i.ibb.co/SNRwr06/choose.png" alt="choose" border="0"></center>

In [ ]:
#@title 4. Stylize image
#@markdown If you are running this demo on a mobile device, it might take between 1-2 minutes to stylize 'big' image so just wait till you see the resulting image
content_blending_ratio = 0 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown You're encouraged to play with the different values of `content_blending_ratio`.


#@markdown ***If the process crashes it's probably due the size of the image you chose so just re-run the demo and try to pick a smaller image.***
content = "/content/content.jpg"
content = load_image(content)
content = np.expand_dims(content,axis=0)
content = content.astype(np.float32)

content_image_height = content.shape[1]
content_image_width = content.shape[2]


def run_style_transform(content_img, path):
  # Load the model.
  interpreter = tf.lite.Interpreter(model_path=path)

  # Set model input.
  input_details = interpreter.get_input_details()
  for index in range(len(input_details)):
    if input_details[index]["name"]=='content':
      index = input_details[index]["index"]
      interpreter.resize_tensor_input(index, [1, content_image_height, content_image_width, 3])
  interpreter.allocate_tensors()

  # Set model inputs.
  for index in range(len(input_details)):
    if input_details[index]["name"]=='content':
      interpreter.set_tensor(input_details[index]["index"], content_img)

  interpreter.invoke()

  # Transform content image.
  stylized_image = interpreter.tensor(interpreter.get_output_details()[0]["index"])()

  return stylized_image

print('Working on it, just wait couple of seconds!')
style = run_style_transform(content, style_transfer_path)

if content_blending_ratio > 0:

  style = tf.image.resize(style, size=(content_image_height, content_image_width))
  style = content_blending_ratio*content + (1.0 - content_blending_ratio)*style


style = np.clip(style, 0, 255)
style = style.astype(np.uint8)

predicted_output = tf.squeeze(style).numpy()
show_image(predicted_output)

plt.imsave('result.png', predicted_output)

In [ ]:
#@title 5. Save your results
#@markdown After the saving, press the image to see it full size and then just download it.


#@markdown If you experiment with the different `content_blending_ratio` values make sure you run this code block again in order to store your results online.

images = [style]
captions = ["stylized_image"]

wandb.init(project='fst', entity='costico', anonymous='allow')
wandb.log({"Results": [wandb.Image(tf.squeeze(image, 0), caption=caption)
    for (image, caption) in zip(images, captions)]})
display(wandb.jupyter.Run())